In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load the data
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [7]:
# Merge customer and transaction data
merged_data = pd.merge(transactions, customers, on="CustomerID", how="left")
merged_data = pd.merge(merged_data, products, on="ProductID", how="left")

# Feature Engineering: Creating customer profiles
# Aggregating transaction data by CustomerID
customer_profile = merged_data.groupby('CustomerID').agg(
    total_spending=('TotalValue', 'sum'),
    transaction_count=('TransactionID', 'count'),
    unique_products=('ProductID', 'nunique')
).reset_index()
customer_profile

,CustomerID,total_spending,transaction_count,unique_products
0,C0001,3354.52,5,5
1,C0002,1862.74,4,4
2,C0003,2725.38,4,4
3,C0004,5354.88,8,8
4,C0005,2034.24,3,3
...,...,...,...,...
194,C0196,4982.88,4,3
195,C0197,1928.65,3,3
196,C0198,931.83,2,2
197,C0199,1979.28,4,4


In [ ]:
# Scaling features to normalize them
scaler = StandardScaler()
customer_profile[['total_spending', 'transaction_count', 'unique_products']] = scaler.fit_transform(
    customer_profile[['total_spending', 'transaction_count', 'unique_products']]
)

# Drop unnecessary columns before merging
customer_profile = customer_profile.drop(columns=['CustomerName', 'Region', 'SignupDate'])

# Merge the customer profile with the original customer info with custom suffixes
customer_profile = pd.merge(customers, customer_profile, on="CustomerID", how="left", suffixes=('', '_profile'))

# Extracting relevant features: we use `total_spending`, `transaction_count`, and `unique_products` to represent customers
features = customer_profile[['total_spending', 'transaction_count', 'unique_products']]

# Check for null values
null_values = customer_profile.isnull().sum()
print("Null values in each column:\n", null_values)

# Handle null values (example: fill with mean for numerical columns)
customer_profile['total_spending'].fillna(customer_profile['total_spending'].mean(), inplace=True)
customer_profile['transaction_count'].fillna(customer_profile['transaction_count'].mode()[0], inplace=True)
customer_profile['unique_products'].fillna(customer_profile['unique_products'].mode()[0], inplace=True)

# Verify that there are no more null values
null_values_after = customer_profile.isnull().sum()
print("Null values after handling:\n", null_values_after)



Null values in each column:
 CustomerID              0
CustomerName            0
Region                  0
SignupDate              0
CustomerName_profile    0
Region_profile          0
SignupDate_profile      0
CustomerName_x          0
Region_x                0
SignupDate_x            0
CustomerName_y          0
Region_y                0
SignupDate_y            0
total_spending          0
transaction_count       0
unique_products         0
frequency               0
dtype: int64
Null values after handling:
 CustomerID              0
CustomerName            0
Region                  0
SignupDate              0
CustomerName_profile    0
Region_profile          0
SignupDate_profile      0
CustomerName_x          0
Region_x                0
SignupDate_x            0
CustomerName_y          0
Region_y                0
SignupDate_y            0
total_spending          0
transaction_count       0
unique_products         0
frequency               0
dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_5008\2994487845.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_profile['total_spending'].fillna(customer_profile['total_spending'].mean(), inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5008\2994487845.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

In [26]:
customer_profile

,CustomerID,CustomerName,Region,SignupDate,CustomerName_profile,Region_profile,SignupDate_profile,CustomerName_x,Region_x,SignupDate_x,CustomerName_y,Region_y,SignupDate_y,total_spending,transaction_count,unique_products,frequency
0,C0001,Lawrence Carroll,South America,2022-07-10,Lawrence Carroll,South America,2022-07-10,Lawrence Carroll,South America,2022-07-10,Lawrence Carroll,South America,2022-07-10,-0.061856,-0.011487,0.050172,4
1,C0002,Elizabeth Lutz,Asia,2022-02-13,Elizabeth Lutz,Asia,2022-02-13,Elizabeth Lutz,Asia,2022-02-13,Elizabeth Lutz,Asia,2022-02-13,-0.879946,-0.468667,-0.425269,4
2,C0003,Michael Rivera,South America,2024-03-07,Michael Rivera,South America,2024-03-07,Michael Rivera,South America,2024-03-07,Michael Rivera,South America,2024-03-07,-0.406876,-0.468667,-0.425269,6
3,C0004,Kathleen Rodriguez,South America,2022-10-09,Kathleen Rodriguez,South America,2022-10-09,Kathleen Rodriguez,South America,2022-10-09,Kathleen Rodriguez,South America,2022-10-09,1.035138,1.360054,1.476494,7
4,C0005,Laura Weber,Asia,2022-08-15,Laura Weber,Asia,2022-08-15,Laura Weber,Asia,2022-08-15,Laura Weber,Asia,2022-08-15,-0.785896,-0.925848,-0.900709,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,C0196,Laura Watts,Europe,2022-06-07,Laura Watts,Europe,2022-06-07,Laura Watts,Europe,2022-06-07,Laura Watts,Europe,2022-06-07,0.831134,-0.468667,-0.900709,4
196,C0197,Christina Harvey,Europe,2023-03-21,Christina Harvey,Europe,2023-03-21,Christina Harvey,Europe,2023-03-21,Christina Harvey,Europe,2023-03-21,-0.843801,-0.925848,-0.900709,5
197,C0198,Rebecca Ray,Europe,2022-02-27,Rebecca Ray,Europe,2022-02-27,Rebecca Ray,Europe,2022-02-27,Rebecca Ray,Europe,2022-02-27,-1.390456,-1.383028,-1.376150,5
198,C0199,Andrea Jenkins,Europe,2022-12-03,Andrea Jenkins,Europe,2022-12-03,Andrea Jenkins,Europe,2022-12-03,Andrea Jenkins,Europe,2022-12-03,-0.816036,-0.468667,-0.425269,3


I have built the model using cosine similarity because cosine similarity is better suited for this data compared to other distance metrics like Euclidean distance or Manhattan distance because it focuses on measuring the similarity in patterns of behavior rather than the absolute differences in data. In this case, customers have varying transaction volumes, spending habits, and product preferences, making their data highly sparse and unevenly distributed. While Euclidean or Manhattan distance would be sensitive to differences in the magnitude of these values (e.g., one customer spending more than another), cosine similarity effectively compares customers based on the direction of their behavior vectors, ignoring the scale. This makes cosine similarity more resilient to sparsity and scale differences, enabling it to better capture similar behavior even when the absolute spending or transaction count varies significantly between customers.

In [27]:
cosine_sim = cosine_similarity(features)

In [28]:
# Function to get top 3 lookalike customers based on cosine similarity
def get_top_lookalikes(customer_id, num_lookalikes=3):
    customer_index = customer_profile[customer_profile['CustomerID'] == customer_id].index[0]
    similarity_scores = cosine_sim[customer_index]
    similar_indices = similarity_scores.argsort()[-(num_lookalikes + 1):-1]  # Exclude self similarity
    similar_customers = customer_profile.iloc[similar_indices]
    similar_customers['score'] = similarity_scores[similar_indices]
    return similar_customers[['CustomerID', 'score']]


In [29]:
# Create a list of top 3 lookalikes for each of the first 20 customers
lookalike_data = []
for customer_id in customer_profile['CustomerID'][:20]:
    top_lookalikes = get_top_lookalikes(customer_id)
    lookalike_data.append({
        'cust_id': customer_id,
        'lookalikes': top_lookalikes.to_dict(orient='records')
    })

C:\Users\Admin\AppData\Local\Temp\ipykernel_5008\3034511774.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_customers['score'] = similarity_scores[similar_indices]
C:\Users\Admin\AppData\Local\Temp\ipykernel_5008\3034511774.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_customers['score'] = similarity_scores[similar_indices]
C:\Users\Admin\AppData\Local\Temp\ipykernel_5008\3034511774.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [32]:
# Save the lookalikes and their scores into a DataFrame
lookalike_df = pd.DataFrame(lookalike_data)

# Save to CSV file
lookalike_df.to_csv('Lookalike.csv', index=False)

# Display the result
print(lookalike_df.head())

  cust_id                                         lookalikes
0   C0001  [{'CustomerID': 'C0056', 'score': 0.9304265739...
1   C0002  [{'CustomerID': 'C0010', 'score': 0.9991818003...
2   C0003  [{'CustomerID': 'C0131', 'score': 0.9995698787...
3   C0004  [{'CustomerID': 'C0108', 'score': 0.9993154296...
4   C0005  [{'CustomerID': 'C0095', 'score': 0.9999466322...
